In [1]:
import torch
import torch.nn as nn
from torch import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print("ix: ", ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

ix:  tensor([175853,  52046, 135846,   4439])
inputs:
tensor([[68,  1, 68, 67, 58,  1, 56, 68],
        [58, 72, 72, 11,  1, 47, 61, 58],
        [57,  1, 73, 61, 58,  0, 47, 62],
        [ 1,  1,  1,  1,  1,  1,  1,  1]], device='cuda:0')
targets:
tensor([[ 1, 68, 67, 58,  1, 56, 68, 74],
        [72, 72, 11,  1, 47, 61, 58, 67],
        [ 1, 73, 61, 58,  0, 47, 62, 79],
        [ 1,  1,  1,  1,  1,  1,  1,  1]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets):
        logits = self.token_embedding_table(index)

        return logits